# NPS - опрос

Заказчик этого исследования — большая телекоммуникационная компания, которая оказывает услуги на территории всего СНГ. Перед компанией стоит задача определить текущий уровень потребительской лояльности, или NPS (от англ. Net Promoter Score), среди клиентов из России.

Прежде чем начать работу, необходимо получить доступ к базе данных.

Далее нужно подготовить данные для дашборда. Преобразовывать данные необходимо с помощью SQL-запросов.
Нужно собрать в одну витрину данные из разных таблиц. Эту витрину мы будеем использовать для дашборда.

Напишем запрос, который выгрузит следующие поля:
- `user_id`	Идентификатор клиента
- `lt_day`	Количество дней «жизни» клиента
- `is_new`	Поле хранит информацию о том, является ли клиент новым
- `age`	Возраст
- `gender_segment`	Пол
- `os_name`	Тип операционной системы
- `cpe_type_name`	Тип устройства
- `country`	Страна проживания
- `city`	Город проживания
- `age_segment`	Возрастной сегмент
- `traffic_segment`	Сегмент по объёму потребляемого трафика
- `lifetime_segment`	Сегмент по количеству дней «жизни»
- `nps_score`	Оценка клиента в NPS-опросе
- `nps_group`	Поле хранит информацию о том, к какой группе относится оценка клиента в опросе

Определить, является ли клиент новым, можно по количеству дней «жизни». Такой показатель ещё называют лайфтайм. Его отсчитывают с момента совершения важного события для бизнеса, например покупки сим-карты для телеком-компании. Клиента считают новым, если количество дней «жизни» составляет не более 365 дней.

Получившуюся таблицу сохраним как .csv файл.

In [1]:
import pandas as pd
from sqlalchemy import create_engine

path_to_db = 'datasets/telecomm_csi.db' # путь к файлу
engine = create_engine(f'sqlite:///{path_to_db}', echo = False)

query = """ 
WITH
u AS (SELECT *,
                (CASE 
                    WHEN nps_score <= 6 THEN "detractors"
                    WHEN nps_score > 6 and nps_score <= 8 THEN "passives"
                    WHEN nps_score > 8 THEN "promoters"
                END) AS nps_group,
                (CASE
                    WHEN gender_segment = 0 THEN "мужской"
                    WHEN gender_segment = 1 THEN "женский"
                END) AS gender,
                (CASE
                    WHEN lt_day <= 365 THEN "новый"
                    WHEN lt_day > 365 THEN "старый"
                END) AS is_new
        FROM user),
l AS (SELECT *
     FROM location),
a_s AS (SELECT *
     FROM age_segment),
ts AS (SELECT *
     FROM traffic_segment),
ls AS (SELECT *
     FROM lifetime_segment)
SELECT u.user_id,
       u.lt_day,
       u.is_new,
       u.age,
       u.gender,
       u.os_name,
       u.cpe_type_name,
       l.country,
       l.city,
       a_s.bucket_min AS age_segment_min,
       a_s.bucket_max AS age_segment_max,
       a_s.title AS age_segment,
       ts.bucket_min AS traffic_segment_min,
       ts.bucket_max AS traffic_segment_max,
       ts.title AS traffic_segment,
       ls.bucket_min AS lifetime_segment_min,
       ls.bucket_max AS lifetime_segment_max,
       ls.title AS lifetime_segment,
       u.nps_score,
       u.nps_group
FROM u
JOIN l
ON u.location_id = l.location_id
JOIN a_s
ON u.age_gr_id = a_s.age_gr_id
JOIN ts
ON u.tr_gr_id = ts.tr_gr_id
JOIN ls
ON u.lt_gr_id = ls.lt_gr_id
"""

df = pd.read_sql(query, engine) 

In [2]:
df.to_csv('df.csv', index = False)
df.head(5)

,user_id,lt_day,is_new,age,gender,os_name,cpe_type_name,country,city,age_segment_min,age_segment_max,age_segment,traffic_segment_min,traffic_segment_max,traffic_segment,lifetime_segment_min,lifetime_segment_max,lifetime_segment,nps_score,nps_group
0,A001A2,2320,старый,45.0,женский,ANDROID,SMARTPHONE,Россия,Уфа,45.0,54.0,05 45-54,1.0,5.0,04 1-5,36.0,NaN,08 36+,10,promoters
1,A001WF,2344,старый,53.0,мужской,ANDROID,SMARTPHONE,Россия,Киров,45.0,54.0,05 45-54,1.0,5.0,04 1-5,36.0,NaN,08 36+,10,promoters
2,A003Q7,467,старый,57.0,мужской,ANDROID,SMARTPHONE,Россия,Москва,55.0,64.0,06 55-64,20.0,25.0,08 20-25,13.0,24.0,06 13-24,10,promoters
3,A004TB,4190,старый,44.0,женский,IOS,SMARTPHONE,Россия,РостовнаДону,35.0,44.0,04 35-44,0.1,1.0,03 0.1-1,36.0,NaN,08 36+,10,promoters
4,A004XT,1163,старый,24.0,мужской,ANDROID,SMARTPHONE,Россия,Рязань,16.0,24.0,02 16-24,5.0,10.0,05 5-10,36.0,NaN,08 36+,10,promoters


Используя получившийся .csv файл, построим дашборд на платформе Tableau Public

### Ссылка на дашборд на сайте Tableau Public:

https://public.tableau.com/app/profile/.71704574/viz/practicum_prefab_2/_NPS?publish=yes

### Ссылка на pdf-файл с презентацией:

https://disk.yandex.ru/i/zCxHA83sBUp9Kg